**Libraries**

In [ ]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import importlib
import sys

sys.path.append("../")
from modules import common, chapter_01, chapter_08

importlib.reload(common)
importlib.reload(chapter_01)
importlib.reload(chapter_08)

**Simple convolution network**

In [ ]:
mnist = keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
x = keras.layers.Rescaling(1.0 / 255)(inputs)
x = keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.5)(x)
outputs = keras.layers.Dense(units=10, activation=tf.nn.softmax)(x)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.TensorBoard(log_dir="../resources/logs"),
    keras.callbacks.ModelCheckpoint(
        filepath="../resources/models/tmp/model.keras",
        monitor="val_sparse_categorical_accuracy",
        save_best_only=True,
    ),
]
history = model.fit(
    x=x_train,
    y=y_train,
    validation_split=0.3,
    epochs=30,
    callbacks=callbacks,
    verbose=False,
    batch_size=512,
)
model.evaluate(x_test, y_test)

**Cats and dog dataset**

In [ ]:
# -------------------------------
# Dataset
# -------------------------------
new_base_dir, train_dataset, val_dataset, test_dataset = chapter_08.dataset_batches()
# -------------------------------
# Augmentation Layers
# -------------------------------
augmentation = keras.Sequential(
    [
        keras.layers.RandomFlip(mode="horizontal"),
        keras.layers.RandomZoom(height_factor=0.8),
        keras.layers.RandomTranslation(height_factor=0.8, width_factor=0.8),
    ]
)

**Training from scratch without augmentation**

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = keras.layers.Rescaling(1.0 / 255)(inputs)
x = keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.5)(x)
outputs = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.TensorBoard(log_dir="../resources/logs"),
    keras.callbacks.ModelCheckpoint(
        filepath="../resources/models/tmp/model.keras",
        monitor="val_binary_accuracy",
        save_best_only=True,
    ),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=callbacks,
    verbose=False,
    batch_size=1024,
)
model.evaluate(x_test, y_test)

**Training from scratch with augmentation**

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = augmentation(inputs)
x = keras.layers.Rescaling(1.0 / 255)(x)
x = keras.layers.Conv2D(filters=32, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=64, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=128, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.MaxPool2D(pool_size=2)(x)
x = keras.layers.Conv2D(filters=256, kernel_size=3, activation=tf.nn.relu)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dropout(rate=0.5)(x)
outputs = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.TensorBoard(log_dir="../resources/logs"),
    keras.callbacks.ModelCheckpoint(
        filepath="../resources/models/tmp/model.keras",
        monitor="val_binary_accuracy",
        save_best_only=True,
    ),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=callbacks,
    verbose=False,
    batch_size=1024,
)
model.evaluate(x_test, y_test)

**Training with feature extraction without augmentation**

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(180, 180, 3),
)

In [ ]:
def extract_features(dataset):
    features, labels = [], []
    for images_batch, labels_batch in dataset:
        images_batch = keras.applications.vgg16.preprocess_input(x=images_batch)
        features_batch = conv_base.predict(x=images_batch, verbose=False)
        features.append(features_batch), labels.append(labels_batch)
    return np.concatenate(features), np.concatenate(labels)

In [ ]:
x_train, y_train = extract_features(train_dataset)
x_val, y_val = extract_features(val_dataset)
x_test, y_test = extract_features(test_dataset)

In [ ]:
inputs = keras.Input(shape=(5, 5, 512))
x = keras.layers.Flatten()(inputs)
x = keras.layers.Dense(units=256, activation=None)(x)
x = keras.layers.Dropout(rate=0.5)(x)
outputs = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.TensorBoard(log_dir="../resources/logs"),
    keras.callbacks.ModelCheckpoint(
        filepath="../resources/models/tmp/model.keras",
        monitor="val_binary_accuracy",
        save_best_only=True,
    ),
]
model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_val, y_val),
    epochs=30,
    callbacks=callbacks,
    verbose=False,
    batch_size=1024,
)
model.evaluate(x_test, y_test)

**Training with feature extraction with augmentation**

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
)
conv_base.trainable = False

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(units=256, activation=None)(x)
x = keras.layers.Dropout(rate=0.5)(x)
outputs = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.TensorBoard(log_dir="../resources/logs"),
    keras.callbacks.ModelCheckpoint(
        filepath="../resources/models/tmp/model.keras",
        monitor="val_binary_accuracy",
        save_best_only=True,
    ),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=callbacks,
    verbose=False,
    batch_size=1024,
)
model.evaluate(x_test, y_test)

**Training with fine tuning with augmentation**

In [ ]:
conv_base = keras.applications.vgg16.VGG16(
    weights="imagenet",
    include_top=False,
)
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = augmentation(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(units=256, activation=None)(x)
x = keras.layers.Dropout(rate=0.5)(x)
outputs = keras.layers.Dense(units=1, activation=tf.nn.sigmoid)(x)

In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[keras.metrics.BinaryAccuracy()],
    optimizer=keras.optimizers.legacy.RMSprop(),
)
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
    keras.callbacks.TensorBoard(log_dir="../resources/logs"),
    keras.callbacks.ModelCheckpoint(
        filepath="../resources/models/tmp/model.keras",
        monitor="val_binary_accuracy",
        save_best_only=True,
    ),
]
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30,
    callbacks=callbacks,
    verbose=False,
    batch_size=1024,
)
model.evaluate(x_test, y_test)